In [417]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools
from sklearn.pipeline import FeatureUnion
import matplotlib.pyplot as plt

%matplotlib inline

This research aimed at the case of customers default payments in Taiwan and compares the predictive accuracy of probability of default among six data mining methods.

In [418]:
df = pd.read_csv('credit_card_clients.csv', sep=';')
df.head(10)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
5,6,50000,1,1,2,37,0,0,0,0,...,19394,19619,20024,2500,1815,657,1000,1000,800,0
6,7,500000,1,1,2,29,0,0,0,0,...,542653,483003,473944,55000,40000,38000,20239,13750,13770,0
7,8,100000,2,2,2,23,0,-1,-1,0,...,221,-159,567,380,601,0,581,1687,1542,0
8,9,140000,2,3,1,28,0,0,2,0,...,12211,11793,3719,3329,0,432,1000,1000,1000,0
9,10,20000,1,3,2,35,-2,-2,-2,-2,...,0,13007,13912,0,0,0,13007,1122,0,0


In [419]:
df.rename(columns={'default payment next month': 'defolts'}, inplace=True)
df.drop(columns=['ID'])

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,defolts
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000,1,3,1,39,0,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,150000,1,3,2,43,-1,-1,-1,-1,0,...,8979,5190,0,1837,3526,8998,129,0,0,0
29997,30000,1,2,2,37,4,3,2,-1,0,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,80000,1,3,1,41,1,-1,0,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [420]:
df.head(10)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,defolts
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
5,6,50000,1,1,2,37,0,0,0,0,...,19394,19619,20024,2500,1815,657,1000,1000,800,0
6,7,500000,1,1,2,29,0,0,0,0,...,542653,483003,473944,55000,40000,38000,20239,13750,13770,0
7,8,100000,2,2,2,23,0,-1,-1,0,...,221,-159,567,380,601,0,581,1687,1542,0
8,9,140000,2,3,1,28,0,0,2,0,...,12211,11793,3719,3329,0,432,1000,1000,1000,0
9,10,20000,1,3,2,35,-2,-2,-2,-2,...,0,13007,13912,0,0,0,13007,1122,0,0


In [421]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['defolts']), df['defolts'], random_state=0)

#### pipeline из прошлого задания

In [422]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля

class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [423]:
continuous_columns = ['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4',
                     'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
                     'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']

In [424]:
# Теперь нам нужно под каждый признак создать трансформер и объединить их в список 
# (сделаем это в цикле, чтобы не мучиться)

from sklearn.preprocessing import StandardScaler

final_transformers = list()

for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('scaler', StandardScaler())
            ])
    
    final_transformers.append((cont_col, cont_transformer))


In [425]:
# Объединим все это в единый пайплайн
feats = FeatureUnion(final_transformers)
feature_processing = Pipeline([('feats', feats)])

In [426]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [427]:
# Создаем пустой датафрейм для хранения результатов
results_df = pd.DataFrame(columns=['Model', 'Thresholds', 'Precision', 'Recall', 'F1-score', 'roc_auc'])

#### по результатам прошлого задания лучшим оказался LGBMClassifier, его и берем

In [428]:
# Добавим модель
from lightgbm import LGBMClassifier


pipeline_lgbm = Pipeline([
    ('features', feats),
    ('classifier', LGBMClassifier(random_state=42)),
])
# обучим наш пайплайн
pipeline_lgbm.fit(X_train, y_train)
# прогнозы для тестовой выборки
preds = pipeline_lgbm.predict_proba(X_test)[:, 1]
preds[:10]

array([0.61917943, 0.31524395, 0.2233257 , 0.08347249, 0.05453937,
       0.34823708, 0.08127963, 0.16355903, 0.15512241, 0.24488463])

In [429]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))
new_row = pd.DataFrame({'Model': ['LGBMClassifier'],
                        'Thresholds': [thresholds[ix]],
                        'Precision': [precision[ix]],
                        'Recall': [recall[ix]],
                        'F1-score': [fscore[ix]],
                        'roc_auc': [roc_auc]})
results_df = pd.concat([results_df, new_row], ignore_index=True)

Best Threshold=0.269095, F-Score=0.544, Precision=0.515, Recall=0.576


In [430]:
results_df

,Model,Thresholds,Precision,Recall,F1-score,roc_auc
0,LGBMClassifier,0.269095,0.515068,0.57598,0.543824,0.807481


In [431]:
df['target'] = np.where(df['defolts'] == 1, 1, -1)
# Разделение на множества P и U
positive_examples = df[df['defolts'] == 1]
positive_P, positive_U = train_test_split(positive_examples, test_size=0.75, random_state=42)
negative_examples = df[df['defolts'] == 0]


# Random negative sampling
num_negative_samples = len(positive_P)
negative_samples = negative_examples.sample(n=num_negative_samples, random_state=42)
P_prime = pd.concat([positive_P, negative_samples])

# Проверка баланса классов
print(P_prime['defolts'].value_counts())

1    1659
0    1659
Name: defolts, dtype: int64


In [432]:
P_prime.loc[P_prime['target'] == -1, 'target'] = 0

pipeline = Pipeline([
    ('features', feats),
    ('classifier', LGBMClassifier(random_state=42)),
])


pipeline.fit(P_prime.drop(columns=['target', 'defolts']), 
             P_prime['target'])

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('LIMIT_BAL',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='LIMIT_BAL')),
                                                                 ('scaler',
                                                                  StandardScaler())])),
                                                ('SEX',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='SEX')),
                                                                 ('scaler',
                                                                  StandardScaler())])),
                                                ('EDUCATION',
                                                 Pipeline(steps=[('selector',
                                   

In [444]:
# наши прогнозы для тестовой выборки
preds = pipeline.predict_proba(X_test)[:, 1]
preds[:10]

array([0.6 , 0.8 , 0.56, 0.26, 0.28, 0.68, 0.39, 0.5 , 0.25, 0.43])

In [434]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
if np.isnan(fscore[ix]):
    fscore[ix] = 0
    precision[ix] = 0
    recall[ix] = 0
    thresholds[ix] = 1.0
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))
roc_auc = roc_auc_score(y_test, preds)
new_row = pd.DataFrame({'Model': ['LGBMClassifier New'],
                        'Thresholds': [thresholds[ix]],
                        'Precision': [precision[ix]],
                        'Recall': [recall[ix]],
                        'F1-score': [fscore[ix]],
                        'roc_auc': [roc_auc]})
results_df = pd.concat([results_df, new_row], ignore_index=True)
results_df

Best Threshold=1.000000, F-Score=0.000, Precision=0.000, Recall=0.000


/var/folders/jt/ml43tfs95vn8qnrwr22_cvch0000gp/T/ipykernel_68833/795515919.py:3: RuntimeWarning: invalid value encountered in true_divide
  fscore = (2 * precision * recall) / (precision + recall)


,Model,Thresholds,Precision,Recall,F1-score,roc_auc
0,LGBMClassifier,0.269095,0.515068,0.57598,0.543824,0.807481
1,LGBMClassifier New,1.000000,0.000000,0.00000,0.000000,0.807481


# Метрики все по нулям и NaN - не могу понять где я не так сделал...

In [435]:
mod_data = X_train.copy()
mod_data['Exited'] = y_train
mod_data.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,Exited
12880,12881,130000,2,1,2,27,0,0,0,0,...,34310,21900,780,2000,2000,5000,438,780,0,0
14868,14869,130000,2,3,1,53,0,0,0,0,...,24296,25436,27451,2343,2386,2296,1536,2451,2659,0
14280,14281,140000,1,3,1,38,2,2,2,2,...,135551,135859,134687,22500,0,10000,2600,2300,5300,1
5318,5319,270000,2,2,1,33,-1,-1,-1,-1,...,1556,7952,10794,8874,3567,1556,7952,4000,2000,0
15421,15422,220000,2,2,1,53,2,0,0,0,...,215040,210884,210479,8490,8100,8000,7804,8254,6640,1


In [436]:
# get the indices of the positives samples
pos_ind = mod_data[mod_data['Exited'] == 1].sample(frac=1, random_state=42).index

# leave just 25% of the positives marked
perc = 0.25
pos_sample_len = int(np.ceil(perc * len(pos_ind)))

print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 1251/5004 as positives and unlabeling the rest


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [437]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample, 'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    21249
 1     1251
Name: class_test, dtype: int64


In [438]:
mod_data.head(5)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,Exited,class_test
12880,12881,130000,2,1,2,27,0,0,0,0,...,21900,780,2000,2000,5000,438,780,0,0,-1
14868,14869,130000,2,3,1,53,0,0,0,0,...,25436,27451,2343,2386,2296,1536,2451,2659,0,-1
14280,14281,140000,1,3,1,38,2,2,2,2,...,135859,134687,22500,0,10000,2600,2300,5300,1,-1
5318,5319,270000,2,2,1,33,-1,-1,-1,-1,...,7952,10794,8874,3567,1556,7952,4000,2000,0,-1
15421,15422,220000,2,2,1,53,2,0,0,0,...,210884,210479,8490,8100,8000,7804,8254,6640,1,-1


In [439]:
mod_data = mod_data.sample(frac=1, random_state=42)


data_N = mod_data[mod_data['class_test'] == -1]
data_P = mod_data[mod_data['class_test'] == 1]

neg_sample = data_N[:data_P.shape[0]]
pos_sample = data_P.copy()

print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1, random_state=42)

(1251, 26) (1251, 26)


In [440]:
sample_train.loc[sample_train['class_test'] == -1, 'class_test'] = 0

pipeline = Pipeline([
    ('features', feats),
    ('classifier', RandomForestClassifier(random_state=42)),
])


pipeline.fit(sample_train.drop(columns=['class_test', 'Exited']), 
             sample_train['class_test'])

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('LIMIT_BAL',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='LIMIT_BAL')),
                                                                 ('scaler',
                                                                  StandardScaler())])),
                                                ('SEX',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='SEX')),
                                                                 ('scaler',
                                                                  StandardScaler())])),
                                                ('EDUCATION',
                                                 Pipeline(steps=[('selector',
                                   

In [441]:
# наши прогнозы для тестовой выборки
preds = pipeline.predict_proba(X_test)[:, 1]
preds[:10]

array([0.6 , 0.8 , 0.56, 0.26, 0.28, 0.68, 0.39, 0.5 , 0.25, 0.43])

In [442]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))
new_row = pd.DataFrame({'Model': ['pu-learning'],
                        'Thresholds': [thresholds[ix]],
                        'Precision': [precision[ix]],
                        'Recall': [recall[ix]],
                        'F1-score': [fscore[ix]],
                        'roc_auc': [roc_auc]})
results_df = pd.concat([results_df, new_row], ignore_index=True)

Best Threshold=0.990000, F-Score=nan, Precision=0.000, Recall=0.000


/var/folders/jt/ml43tfs95vn8qnrwr22_cvch0000gp/T/ipykernel_68833/1910827782.py:3: RuntimeWarning: invalid value encountered in true_divide
  fscore = (2 * precision * recall) / (precision + recall)


In [443]:
results_df

,Model,Thresholds,Precision,Recall,F1-score,roc_auc
0,LGBMClassifier,0.269095,0.515068,0.57598,0.543824,0.807481
1,LGBMClassifier New,1.000000,0.000000,0.00000,0.000000,0.807481
2,pu-learning,0.990000,0.000000,0.00000,NaN,0.807481


In [445]:
from tqdm import tqdm

metrics_df = pd.DataFrame(columns=['frac', 'thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])


for frac in tqdm(np.linspace(0.1, 1, 9)):
    mod_data = X_train.copy()
    mod_data['Exited'] = y_train
    mod_data.head()

    # get the indices of the positives samples
    pos_ind = mod_data[mod_data['Exited'] == 1].sample(frac=1, random_state=42).index

    pos_sample_len = int(np.ceil(frac * len(pos_ind)))
    pos_sample = pos_ind[:pos_sample_len]
    
    mod_data['class_test'] = -1
    mod_data.loc[pos_sample,'class_test'] = 1
    
    mod_data = mod_data.sample(frac=1, random_state=42)


    data_N = mod_data[mod_data['class_test'] == -1]
    data_P = mod_data[mod_data['class_test'] == 1]

    neg_sample = data_N[:data_P.shape[0]]
    pos_sample = data_P.copy()

    sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1, random_state=42)

    sample_train.loc[sample_train['class_test'] == -1, 'class_test'] = 0

    pipeline = Pipeline([
        ('features', feats),
        ('classifier', RandomForestClassifier(random_state=42)),
    ])

    pipeline.fit(sample_train.drop(columns=['class_test','Exited']), 
                 sample_train['class_test'])
    
    # наши прогнозы для тестовой выборки
    preds = pipeline.predict_proba(X_test)[:, 1]
    preds[:10]

    precision, recall, thresholds = precision_recall_curve(y_test, preds)

    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    roc_auc = roc_auc_score(y_test, preds)

    metrics_df = metrics_df.append({
        'frac': frac,
        'thresh': thresholds[ix],
        'F-Score': fscore[ix],
        'Precision': precision[ix],
        'Recall': recall[ix],
        'ROC AUC': roc_auc
    }, ignore_index=True)

metrics_df

  0%|                                                     | 0/9 [00:00<?, ?it/s]/var/folders/jt/ml43tfs95vn8qnrwr22_cvch0000gp/T/ipykernel_68833/2184144724.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({
 11%|█████                                        | 1/9 [00:00<00:02,  3.32it/s]/var/folders/jt/ml43tfs95vn8qnrwr22_cvch0000gp/T/ipykernel_68833/2184144724.py:47: RuntimeWarning: invalid value encountered in true_divide
  fscore = (2 * precision * recall) / (precision + recall)
/var/folders/jt/ml43tfs95vn8qnrwr22_cvch0000gp/T/ipykernel_68833/2184144724.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({
 22%|██████████                                   | 2/9 [00:00<00:02,  2.60it/s]/var/folders/jt/ml43tfs95vn8qnrwr22_cvch0000gp/T/ipykernel_

,frac,thresh,F-Score,Precision,Recall,ROC AUC
0,0.1000,0.550000,0.499096,0.431443,0.591912,0.752945
1,0.2125,0.990000,NaN,0.000000,0.000000,0.757969
2,0.3250,0.590000,0.507409,0.500896,0.514093,0.754113
3,0.4375,0.540000,0.515837,0.456000,0.593750,0.760090
4,0.5500,0.550000,0.517609,0.466765,0.580882,0.760965
5,0.6625,0.570000,0.517396,0.477226,0.564951,0.762823
6,0.7750,0.508333,0.515433,0.436464,0.629289,0.763239
7,0.8875,0.613333,0.515786,0.526484,0.505515,0.764349
8,1.0000,0.560833,0.525865,0.492769,0.563725,0.772358
